In [1]:
from Helper_Utilities import *

*Summary Buffer Memory*

In [19]:
from langchain.memory import ConversationSummaryBufferMemory
memory = ConversationSummaryBufferMemory(
    llm=llm, max_token_limit=10, return_messages=True
)
memory.save_context({"input": "xin chào , tôi là Dang, là kĩ sư cơ khí."}, {"output": "chào bạn, tôi có thể giúp gì cho bạn hôm nay"})
memory.save_context({"input": "tôi cần tìm sách Cẩm nang cơ khí"}, {"output": "cuốn sách đó nằm ở kệ số 6"})
messages = memory.chat_memory.messages
previous_summary = ""
memory.predict_new_summary(messages, previous_summary)

'\nThe human asks the AI what its purpose is. The AI responds that its purpose is to assist and improve human lives.'

*Knowledge Graph Memory*

In [20]:
from langchain.memory import ConversationKGMemory
from langchain_openai import OpenAI
llm = OpenAI(temperature=0)
memory = ConversationKGMemory(llm=llm)
memory.save_context({"input": "xin chào , tôi là Dang, là kĩ sư cơ khí."}, {"output": "chào bạn, tôi có thể giúp gì cho bạn hôm nay"})
memory.save_context({"input": "tôi cần tìm sách Cẩm nang cơ khí"}, {"output": "cuốn sách đó nằm ở kệ số 6"})

memory.load_memory_variables({"input": "Dang là ai?"})

{'history': 'On Dang: Dang is a kĩ sư cơ khí.'}

*Entity Memory*

In [2]:
from langchain.memory import ConversationEntityMemory
memory = ConversationEntityMemory(llm=llm)
_input = {"input": "xin chào , tôi là Dang, là kĩ sư cơ khí."}
memory.load_memory_variables(_input)
memory.save_context(
    _input,
    {"output": "chào bạn, tôi có thể giúp gì cho bạn hôm nay"}
)
memory.save_context({"input": "tôi cần tìm sách Cẩm nang cơ khí"}, {"output": "cuốn sách đó nằm ở kệ số 6"})
memory.load_memory_variables({"input": "Cẩm nang cơ khí ở đâu?"})

{'history': 'Human: xin chào , tôi là Dang, là kĩ sư cơ khí.\nAI: chào bạn, tôi có thể giúp gì cho bạn hôm nay\nHuman: tôi cần tìm sách Cẩm nang cơ khí\nAI: cuốn sách đó nằm ở kệ số 6',
 'entities': {'Cẩm nang cơ khí': ''}}

*custom memory*

In [26]:
from Helper_Utilities import *
interrupt_function_def = {
    "name": "route",
    "description":"Help to give a hint to the input",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "messages": {
                "title": "messages",
                "output": 'text',
            }
        },
        "required": ["messages"],
    },
}
memory_direct_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", (MEMORY_DIRECT_PROMPT)),
        # MessagesPlaceholder(variable_name="history"),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
memory_direct_chain = (
    memory_direct_prompt 
    | llm.bind_functions(functions=[interrupt_function_def], function_call="route")
    | JsonOutputFunctionsParser()
)

In [29]:
OpenAIHistoryConversation = [HumanMessage(content='tôi là Dang, là một kĩ sư cơ khí, thư viện có sách về cơ khí không'),
AIMessage(content='Đã tìm thấy sách cơ khí bạn cần , đó là cuốn Cẩm nang cơ khí'),
HumanMessage(content = "tôi muôn trả sách"),
AIMessage(content = "Sách đã trả: 'Lập trình python căn bản' . Quá trình trả sách đã hoàn tất, chúc bạn một ngày tốt lành"),
HumanMessage(content = 'cuốn sách đó có hay không?')]

In [30]:
memory_direct_chain.invoke({"messages":OpenAIHistoryConversation})

{'messages': ['tôi muốn biết cuốn sách Lập trình python căn bản ở kệ nào?']}